# Velocity & Vorticity

### Operational Code

In [ ]:
# code adapted from Raf Dussin
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import animation
import cartopy.crs as ccrs
from datetime import datetime
from matplotlib import cm
import os
from xgcm import Grid
import glob
from PIL import Image
import dask
import sys
import matplotlib.image as image
dask.config.set(**{'array.slicing.split_large_chunks': False})

#load bluemarble image
pngDir='/glade/u/home/jsimkins/logos/'
img = plt.imread(f'{pngDir}bluemarble.png')
img_extent = (-180, 180, -90, 90)
logo1 = image.imread(f'{pngDir}noaa.png')
logo2 = image.imread(f'{pngDir}gfdl.png')
logo3 = image.imread(f'{pngDir}rutgers.png')
logo4 = image.imread(f'{pngDir}esmg.png')



workdir='/glade/work/jsimkins/runs-MOM6/nwa25/'
# grab experiment name
with open(workdir + 'diag_table') as f:
    expName = f.readline()[:-1]

# grab file
try:
    dailyOut = sys.argv[1]
    staticOut = sys.argv[2]
except:
    print("Add the files we're using as an argument!")
        

var_list = ['velocity', 'vorticity']
outDir = 'velvort_images/'

#Crunch the numbers
ds_grid_raw = xr.open_dataset(f'{workdir}{staticOut}')
ds = xr.open_dataset(f'{workdir}{dailyOut}')


ds_grid = ds_grid_raw.drop(labels='time')
grid = Grid(ds_grid, coords={'X': {'center': 'xh', 'outer': 'xq'},
                             'Y': {'center': 'yh', 'outer': 'yq'} }, periodic=[])

speed2 = (grid.interp(ds.ssu, 'X', boundary='fill') * grid.interp(ds.ssu, 'X', boundary='fill') ) + \
         (grid.interp(ds.ssv, 'Y', boundary='fill') * grid.interp(ds.ssv, 'Y', boundary='fill') )

for var in var_list:
    if var == 'velocity':
        for kt in range(len(ds.time.values)):
            print("vel" + str(kt))
            timestr=datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d%H%M%S")
            print(var)
            fileout = f'{workdir}{outDir}/{var}_{timestr}.png'
            if not os.path.exists(fileout):
                fig = plt.figure(figsize=[12.8,7.2])
                left = 0.01
                bottom = 0.01
                width = 0.99
                height = 0.99
                ax1 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
                #ax.add_wmts(url, 'BlueMarble')
                ax1.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(), vmin=0,vmax=255)
                field = ax1.contourf(ds_grid['geolon'], ds_grid['geolat'], np.sqrt(speed2.isel({'time': kt})), np.arange(0., 1.05, 0.05), cmap=cm.Blues_r, extend='both' )#, np.arange(-2,38.5,0.5), cmap=cm.gist_ncar)
                cb = plt.colorbar(field, ticks=[0, 0.25, 0.5, 0.75, 1.0])
                cb.ax.yaxis.set_tick_params(color='w')
                gl = ax1.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
                gl.top_labels = False
                gl.left_labels = False
                gl.right_labels=True
                gl.bottom_labels=True
                gl.xlines = True
                gl.ylines = True
                gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
                gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
                gl.xformatter = LONGITUDE_FORMATTER
                gl.yformatter = LATITUDE_FORMATTER
                gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
                ax1.imshow(logo1, origin='upper', extent=(-99, -95.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo2, origin='upper', extent=(-95, -91.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo3, origin='upper', extent=(-91, -87.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax1.imshow(logo4, origin='upper', extent=(-87, -83.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                plt.title(str("MOM6 NWA25 Velocity: " + ds['time'].isel(time=kt).dt.strftime("%Y-%m-%d").values), fontsize=16, color='w')
                plt.savefig(f'{fileout}', facecolor='black', format='png', bbox_inches='tight', dpi=90)
                plt.close()

    if var == "vorticity":
        ds_grid = ds_grid_raw.isel({'xq': slice(0,-1), 'yq': slice(0,-1)}).drop(labels='time')
        grid = Grid(ds_grid, coords={'X': {'center': 'xh', 'left': 'xq'},
                                    'Y': {'center': 'yh', 'left': 'yq'} }, periodic=[])
        vorticity = (-grid.diff(ds.ssu * ds_grid.dxCu, 'Y', boundary='fill') + grid.diff(ds.ssv * ds_grid.dyCv, 'X', boundary='fill'))/ds_grid.areacello_bu
        #vorticity = (-grid.diff(ds.ssu * ds_grid.dxCu, 'Y', boundary='fill') + grid.diff(ds.v * ds_grid.dyCv, 'X', boundary='fill'))/ds_grid.areacello_bu
        vortplt = vorticity * 1e+4

        for kt in range(len(ds.time.values)):
            print("vort" + str(kt))
            timestr=datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d%H%M%S")
            fileout = f'{workdir}{outDir}/{var}_{timestr}.png'
            if not os.path.exists(fileout):
                fig = plt.figure(figsize=[12.8,7.2])
                left = 0.01
                bottom = 0.01
                width = 0.99
                height = 0.99 #0.95
                ax2 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
                ax2.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(), vmin=0,vmax=255)
                field = ax2.contourf(ds_grid['geolon_c'], ds_grid['geolat_c'], vortplt.isel({'time': kt}), np.arange(-0.5,0.505,0.05), cmap=cm.bwr, extend='both' )#, np.arange(-2,38.5,0.5), cmap=cm.gist_ncar)
                cb = plt.colorbar(field, ticks=[-0.5, -0.25, 0, 0.25, 0.5])
                cb.ax.yaxis.set_tick_params(color='w')
                cb.set_label('$\zeta / f$', color='w', fontsize=18)
                gl = ax2.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
                gl.top_labels = False
                gl.left_labels = False
                gl.right_labels=True
                gl.bottom_labels=True
                gl.xlines = True
                gl.ylines = True
                gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
                gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
                gl.xformatter = LONGITUDE_FORMATTER
                gl.yformatter = LATITUDE_FORMATTER
                gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
                plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w', fontsize=14)
                plt.title(str("MOM6 NWA25 Vorticity: " + ds['time'].isel(time=kt).dt.strftime("%Y-%m-%d").values), fontsize=16, color='w')
                ax2.imshow(logo1, origin='upper', extent=(-99, -95.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo2, origin='upper', extent=(-95, -91.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo3, origin='upper', extent=(-91, -87.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())
                ax2.imshow(logo4, origin='upper', extent=(-87, -83.5, 54.5, 58),zorder=10,transform=ccrs.PlateCarree())

                #plt.savefig(f'vort_frame_{frame}.png',  facecolor='black', format='png', dpi=300)
                plt.savefig(f'{fileout}', facecolor='black', bbox_inches='tight', format='png', dpi=90)
                plt.close()

    img_names = sorted(glob.glob(f'{workdir}{outDir}/*.png'))
    dur_vals = []
    for i in range(0,len(img_names)):
        if i != len(img_names)-1:
            dur_vals.append(200)
    dur_vals.append(3000)

    img, *imgs = [Image.open(f) for f in img_names]
    img.save(fp=f'{workdir}/gifs/{expName}_{dailyOut[0:4]}{var}.gif', format='GIF', append_images=imgs,
         save_all=True, duration=dur_vals, loop=0)



In [24]:

var_list = ['velocity', 'vorticity']
outDir = dict({'velocity':'vel_img', 'vorticity':'vort_img'})
outDir[var]

NameError: name 'var' is not defined

# Velocity Code Testing

In [2]:
# code adapted from Raf Dussin
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import animation
import cartopy.crs as ccrs
from owslib.wms import WebMapService
from matplotlib import cm
import os
from xgcm import Grid
import dask
from datetime import datetime
datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d")
dask.config.set(**{'array.slicing.split_large_chunks': False})

vel_outdir = "/Users/james/Downloads/velocity/"
ds_grid_raw = xr.open_dataset('output/20100505.ocean_5day_epblupdate_soda_obc.nc').isel(z_l=0)
ds = xr.open_mfdataset('output/20100505.ocean_5day_epblupdate_soda_obc.nc').isel(z_l=0)

ds_geo = xr.open_dataset("/Users/james/Documents/Github/esm_lab/grid_gen/gridInfo/nwa25/netcdf3/land_mask.nc")
ds_grid = ds_grid_raw.drop(labels='time')
#ds_grid = ds_grid_raw.isel({'xq': slice(0,-1), 'yq': slice(0,-1)}).drop(labels='time')

grid = Grid(ds_grid, coords={'X': {'center': 'xh', 'outer': 'xq'},
                             'Y': {'center': 'yh', 'outer': 'yq'} }, periodic=[])


speed2 = (grid.interp(ds.u, 'X', boundary='fill') * grid.interp(ds.u, 'X', boundary='fill') ) + \
         (grid.interp(ds.v, 'Y', boundary='fill') * grid.interp(ds.v, 'Y', boundary='fill') ) 


img = plt.imread('/Users/james/Downloads/bluemarble.png')
img_extent = (-180, 180, -90, 90)
for kt in range(len(ds.time.values)):
    print(kt)
    timestr=datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d")
    fileout = f'{outdir}/velocity_{timestr}.png'
    if not os.path.exists(fileout):
        fig = plt.figure(figsize=[12.8,7.2])
        left = 0.01
        bottom = 0.01
        width = 0.99
        height = 0.99
        ax = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
        #ax.add_wmts(url, 'BlueMarble')
        ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
        field = ax.contourf(ds_geo['x'], ds_geo['y'], np.sqrt(speed2.isel({'time': kt})), np.arange(0., 1.05, 0.05), cmap=cm.Blues_r, extend='both' )#, np.arange(-2,38.5,0.5), cmap=cm.gist_ncar)
        cb = plt.colorbar(field, ticks=[0, 0.25, 0.5, 0.75, 1.0])
        cb.ax.yaxis.set_tick_params(color='w')
        gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
        gl.top_labels = False
        gl.left_labels = False
        gl.right_labels=True
        gl.bottom_labels=True
        gl.xlines = True
        gl.ylines = True
        gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
        gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
        gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
        plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
        plt.title(str("MOM6 NWA25 Velocity: " + ds['time'].isel(time=kt).dt.strftime("%Y-%m-%d").values), fontsize=16, color='w')
        plt.savefig(f'{vel_outdir}/{fileout}', facecolor='black', format='png', bbox_inches='tight', dpi=300)
        plt.close()

0
1
2
3
4
5


In [18]:
from datetime import datetime
datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d")

'20100602'

In [19]:
ds['time'].isel(time=kt).dt.strftime("%Y-%m-%d").values

array('2010-06-02', dtype=object)

# Vorticity Code Testing

In [1]:
# code adapted from Raf Dussin
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib import animation
import cartopy.crs as ccrs
#from owslib.wms import WebMapService
from matplotlib import cm
import os
from xgcm import Grid
import glob
from PIL import Image
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False})

vort_outdir = "/home/james/velocity/"
ds_grid_raw = xr.open_mfdataset('*ocean_5day_epblupdate_soda_obc.nc').isel(z_l=0).drop(['temp','salt'])
#ds_grid_raw = xr.open_dataset(f'{timestamp}.ocean_5day_epblupdate_soda_obc.nc').isel(z_l=0)
ds = xr.open_mfdataset(f'*.ocean_5day_epblupdate_soda_obc.nc').isel(z_l=0).drop(['temp','salt'])

#ds_geo = xr.open_dataset("/Users/james/Documents/Github/esm_lab/grid_gen/gridInfo/nwa25/netcdf3/land_mask.nc")
ds_geo = xr.open_dataset("/home/james/gridInfo/nwa25/land_mask.nc")
ds_grid = ds_grid_raw.drop(labels='time')
#ds_grid = ds_grid_raw.isel({'xq': slice(0,-1), 'yq': slice(0,-1)}).drop(labels='time')

grid = Grid(ds_grid, coords={'X': {'center': 'xh', 'outer': 'xq'},
                             'Y': {'center': 'yh', 'outer': 'yq'} }, periodic=[])


speed2 = (grid.interp(ds.u, 'X', boundary='fill') * grid.interp(ds.u, 'X', boundary='fill') ) + \
         (grid.interp(ds.v, 'Y', boundary='fill') * grid.interp(ds.v, 'Y', boundary='fill') ) 

#url = 'http://map1c.vis.earthdata.nasa.gov/wmts-geo/wmts.cgi'
img = plt.imread('bluemarble.png')
img_extent = (-180, 180, -90, 90)
for kt in range(len(ds.time.values)):
    timestr=datetime.strftime(datetime.strptime(str(ds.time.values[kt]), "%Y-%m-%d %H:%M:%S"), "%Y%m%d")
    fileout = f'{outdir}/vorticity_{timestr}.png'
    if not os.path.exists(fileout):
        fig = plt.figure(figsize=[12.8,7.2])
        left = 0.01
        bottom = 0.01
        width = 0.99
        height = 0.99
        ax = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -36, 5, 55], facecolor='black')
        #ax.add_wmts(url, 'BlueMarble')
        ax.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree())
        field = ax.contourf(ds_geo['x'], ds_geo['y'], np.sqrt(speed2.isel({'time': kt})), np.arange(0., 1.05, 0.05), cmap=cm.Blues_r, extend='both' )#, np.arange(-2,38.5,0.5), cmap=cm.gist_ncar)
        cb = plt.colorbar(field, ticks=[0, 0.25, 0.5, 0.75, 1.0])
        cb.ax.yaxis.set_tick_params(color='w')
        gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
        gl.top_labels = False
        gl.left_labels = False
        gl.right_labels=True
        gl.bottom_labels=True
        gl.xlines = True
        gl.ylines = True
        gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
        gl.ylocator = mticker.FixedLocator([10, 20, 30, 40, 50])
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'color': 'darkgreen', 'weight': 'normal'}
        gl.ylabel_style = {'color': 'darkgreen', 'weight': 'normal'}
        plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='w')
        plt.title(str("MOM6 NWA25 Velocity: " + ds['time'].isel(time=kt).dt.strftime("%Y-%m-%d").values), fontsize=16, color='w')
        plt.savefig(f'{fileout}', facecolor='black', format='png', bbox_inches='tight', dpi=300)
        plt.close()

img_names = sorted(glob.glob(vort_outdir + "*.png"))
dur_vals = []
for i in range(0,len(img_names)):
    if i != len(img_names)-1:
        dur_vals.append(200)
dur_vals.append(3000)

img, *imgs = [Image.open(f) for f in img_names]
img.save(fp=vort_outdir + 'vel.gif', format='GIF', append_images=imgs,
     save_all=True, duration=dur_vals, loop=0)

OSError: no files to open